### Import standard GDAL and NumPy packages

In [ ]:
import numpy as np
from osgeo import gdal

### Import SKOPE data analysis functions

In [ ]:
import skope_analysis as sa

### Set the location and name of the data set file to create

In [ ]:
test_dataset_filename =             '../data/annual_5x5x5_dataset_float32_variable.tif'
test_dataset_uncertainty_filename = '../data/annual_5x5x5_dataset_float32_variable_uncertainty.tif'

### Create the test dataset and uncertainty files using a local utility function

In [ ]:
dataset = sa.create_dataset_file(
    filename     = test_dataset_filename,
    format       = 'GTiff',
    pixel_type   = gdal.GDT_Float32, 
    rows         = 5, 
    cols         = 5, 
    bands        = 5,
    origin_x     = -123,
    origin_y     = 45,
    pixel_width  = 1, 
    pixel_height = 1
)

dataset_uncertainty = sa.create_dataset_file(
    filename     = test_dataset_uncertainty_filename,
    format       = 'GTiff',
    pixel_type   = gdal.GDT_Float32, 
    rows         = 5, 
    cols         = 5, 
    bands        = 5,
    origin_x     = -123,
    origin_y     = 45,
    pixel_width  = 1, 
    pixel_height = 1
)

### Define the pixel values to be assigned to the first band of the data set and uncertainties

In [ ]:
import math
nodata = math.nan

band_1_data = np.array([
    [100, 101.1, 102.2, 103.3, 104.4],
    [110, 111.1, 112.2, 113.3, 114.4],
    [120, 121.1, 122.2, 123.3, 124.4],
    [130, 131.1, 132.2, 133.3, nodata],
    [140, 141.1, 142.2, 143.3, 144.4]
])

band_1_uncertainty = np.array([
    [10, 10.1, 10.2, 10.3, 10.4],
    [11, 11.1, 11.2, 11.3, 11.4],
    [12, 12.1, 12.2, 12.3, 12.4],
    [13, 13.1, 13.2, 13.3, 13.4],
    [14, 14.1, 14.2, 14.3, 14.4], 
]) 

### Write pixel values to each band of the data set
For all bands other than the first, the value of each pixel is 100 more than the pixel directly below it in the previous band.

In [ ]:
for i in range(0,5):
    
    sa.write_band(
        dataset = dataset,
        band    = i + 1,
        array   = band_1_data + i * 100,
        nodata  = nodata
    )
    
    sa.write_pixel(dataset,band=i+1,row=3,column=4,value=nodata)

    sa.write_band(
        dataset = dataset_uncertainty,
        band    = i + 1,
        array   = band_1_uncertainty + i * 10,
        nodata  = nodata
    )
    
sa.write_pixel(dataset,band=3,row=2,column=4,value=nodata)
    
dataset.FlushCache()
dataset_uncertainty.FlushCache()

### Confirm some expected pixel values in the new data set and uncertainties

In [ ]:
assert sa.read_pixel(dataset,band=1,row=0,column=0) == 100
assert sa.read_pixel(dataset,band=1,row=4,column=4) - 144.4 < 0.001

assert math.isnan(sa.read_pixel(dataset,band=1,row=3,column=4))
assert math.isnan(sa.read_pixel(dataset,band=2,row=3,column=4))
assert math.isnan(sa.read_pixel(dataset,band=3,row=3,column=4))
assert math.isnan(sa.read_pixel(dataset,band=4,row=3,column=4))
assert math.isnan(sa.read_pixel(dataset,band=5,row=3,column=4))
assert math.isnan(sa.read_pixel(dataset,band=3,row=2,column=4))

### Display pixel values and uncertainties for each band of new data set for easy reference

In [ ]:
for i in range(1,6):
    print("\nband", i, "\n", 
          dataset.GetRasterBand(i).ReadAsArray(), "\n", 
          "\nband", i, "uncertainty\n", 
          dataset_uncertainty.GetRasterBand(i).ReadAsArray()
         )